# ===========================================
# 4_rlhf_experiment.ipynb
# Experimentelles Reinforcement Learning mit menschlichem Feedback (RLHF)
# ===========================================

# Zelle 1: Bibliotheken und Setup
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Experiment läuft auf:", DEVICE)

# Für diesen Experimentellen RLHF-Demo-Ansatz verwenden wir das zuvor trainierte Seq2Seq-Modell.
# Wir simulieren hier menschliches Feedback in Form eines Rewards:
# - "thumbs_up" (Label 1) => Reward = +1.0
# - "thumbs_down" (Label 0) => Reward = 0.0

# Zelle 2: Dummy-Daten und Feedback-Simulation
# Simuliere einen Batch von Beispielen:
# Angenommen, unser Modell generiert zu jedem Beispiel eine Sequenz von Token-IDs.
batch_size = 4
seq_len = 10
vocab_size = 500  # Dummy-Vokabulargröße

# Simuliere Eingaben (Input-Text) als zufällige Token-IDs
dummy_inputs = torch.randint(0, vocab_size, (batch_size, seq_len)).to(DEVICE)

# Simuliere Target-Sequenzen (wir nehmen hier gleiche Länge an)
dummy_targets = torch.randint(0, vocab_size, (batch_size, seq_len)).to(DEVICE)

# Simuliere Feedback-Labels: 1 für positives Feedback, 0 für negatives Feedback
dummy_feedback = torch.tensor([1, 0, 1, 0], dtype=torch.float).to(DEVICE)  # Beispiel-Batch

# Zelle 3: Dummy Seq2Seq-Modell (wie zuvor definiert)
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, pad_idx):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
    
    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.lstm(embedded)
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, pad_idx):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, trg, hidden):
        embedded = self.embedding(trg)
        outputs, hidden = self.lstm(embedded, hidden)
        predictions = self.fc(outputs)
        return predictions, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, pad_idx):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = pad_idx
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        trg_len = trg.size(1)
        vocab_size = self.decoder.fc.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(src.device)
        _, hidden = self.encoder(src)
        input_dec = trg[:, 0].unsqueeze(1)
        for t in range(1, trg_len):
            output, hidden = self.decoder(input_dec, hidden)
            outputs[:, t] = output.squeeze(1)
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(dim=2)
            input_dec = trg[:, t].unsqueeze(1) if teacher_force else top1
        return outputs

# Dummy-Pad-ID (z.B. 0)
pad_id = 0

# Initialisiere das Dummy-Modell
encoder_model = Encoder(vocab_size, embed_dim=64, hidden_dim=128, num_layers=1, pad_idx=pad_id)
decoder_model = Decoder(vocab_size, embed_dim=64, hidden_dim=128, num_layers=1, pad_idx=pad_id)
model = Seq2Seq(encoder_model, decoder_model, pad_id).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Zelle 4: Standard-Loss berechnen (ohne Feedback)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id, reduction="none")
model.train()
outputs = model(dummy_inputs, dummy_targets, teacher_forcing_ratio=0.5)
output_dim = outputs.shape[-1]
outputs_flat = outputs[:, 1:].reshape(-1, output_dim)
trg_flat = dummy_targets[:, 1:].reshape(-1)
loss_tokens = criterion(outputs_flat, trg_flat)
loss_standard = loss_tokens.mean()
print("Standard Loss (ohne Feedback):", loss_standard.item())

# Zelle 5: Feedback-basiertes Fine-Tuning mittels REINFORCE
# Wir nutzen einen vereinfachten REINFORCE-Ansatz: 
# Für jedes Beispiel wird der Gesamtverlust (über die Sequenz) berechnet und
# dann mit dem Feedback-Rewards multipliziert, bevor gemittelt wird.

def compute_weighted_loss(outputs, trg, feedback, criterion, pad_id):
    """
    Berechnet den gewichteten Verlust, wobei pro Beispiel der Verlust (Summe über Tokens)
    mit einem Reward multipliziert wird. Positive Feedbacks erhalten höheren Reward.
    """
    batch_size = outputs.size(0)
    output_dim = outputs.shape[-1]
    # Ignoriere das erste Token (<BOS>)
    outputs = outputs[:, 1:].reshape(-1, output_dim)
    trg = trg[:, 1:].reshape(-1)
    loss_per_token = criterion(outputs, trg)  # [Batch*seq_len]
    # Summe pro Beispiel
    loss_per_example = loss_per_token.view(batch_size, -1).sum(dim=1)  # [Batch]
    # Definiere Reward: positives Feedback (Label 1) => Reward 1.0, negatives (0) => 0.5
    rewards = torch.where(feedback == 1, torch.tensor(1.0, device=DEVICE), torch.tensor(0.5, device=DEVICE))
    weighted_loss = (loss_per_example * rewards).mean()
    return weighted_loss

# Simuliere einen Trainingsschritt mit Feedback
model.train()
optimizer.zero_grad()
outputs = model(dummy_inputs, dummy_targets, teacher_forcing_ratio=0.5)
weighted_loss = compute_weighted_loss(outputs, dummy_targets, dummy_feedback, criterion, pad_id)
weighted_loss.backward()
optimizer.step()
print("Feedback-gewichteter Loss:", weighted_loss.item())

# Zelle 6: Experimentelle Inferenz mit Feedback (ohne direkte RL-Update)
def generate_text(model, input_ids, word2id, id2word, max_length=20, device="cpu"):
    model.eval()
    src_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    _, hidden = encoder_model(src_tensor)
    bos_id = word2id.get("<BOS>", 2)
    input_dec = torch.tensor([[bos_id]], dtype=torch.long).to(device)
    generated_ids = [bos_id]
    with torch.no_grad():
        for _ in range(max_length):
            output, hidden = decoder_model(input_dec, hidden)
            next_id = output.argmax(dim=2).item()
            if next_id == word2id.get("<EOS>", 3):
                break
            generated_ids.append(next_id)
            input_dec = torch.tensor([[next_id]], dtype=torch.long).to(device)
    generated_text = " ".join([id2word.get(i, "<UNK>") for i in generated_ids])
    return generated_text

# Beispiel-Inferenz: Simuliere einen Input
dummy_input = dummy_inputs[0].tolist()  # Nutze einen Dummy-Input aus dem Batch
generated = generate_text(model, dummy_input, { "<BOS>": 2, "<EOS>": 3, "<UNK>": 1 }, { 2: "<BOS>", 3: "<EOS>", 1: "<UNK>" }, max_length=20, device=DEVICE)
print("\nGenerierter Text (Feedback-RLHF-Demo):")
print(generated)

# Zelle 7: Fazit & Ausblick
print("""
Fazit:
- Dieser experimentelle Ansatz zeigt, wie man menschliches Feedback (als Reward) in das Training integrieren kann.
- Durch die Gewichtung des Verlusts werden positive Beispiele stärker gewichtet, was das Modell dazu anregt, qualitativ bessere Texte zu generieren.
- Dies ist ein vereinfachtes Beispiel für Reinforcement Learning from Human Feedback (RLHF).

Nächste Schritte:
- Teste diesen Ansatz mit echten Nutzerdaten und evaluiere die Verbesserungen.
- Experimentiere mit alternativen Reward-Funktionen und erweitere den Ansatz auf Transformer-Modelle (z. B. GPT-2/T5) mit RLHF.
- Integriere fortgeschrittene RL-Methoden (z. B. Policy-Gradient-Ansätze) für eine robustere Optimierung.
""")
